### SoccerNet Object Tracking
#### 02456 Deep learning 2022

Training of YOLOv7

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import os
import time
import yaml
import re
import pandas as pd
import seaborn as sn
from IPython.display import Video
from glob import glob
from io import TextIOWrapper
import collections
import zipfile
import csv
import cv2
from SoccerNet.Downloader import SoccerNetDownloader
from SoccerNet.utils import getListGames

In [ ]:
# download data and unzip them might take a while
mySoccerNetDownloader = SoccerNetDownloader(LocalDirectory="./data")
mySoccerNetDownloader.downloadDataTask(task="tracking", split=["train", "test", "challenge"])

In [ ]:
# clone the repo and download pre-trained weights
!git clone https://github.com/WongKinYiu/yolov7
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-w6_training.pt

In [ ]:
def get_np_array_from_tar_object(tar_extractfl):
     '''converts a buffer from a tar file in np.array'''
     return np.asarray(bytearray(tar_extractfl.read()), dtype=np.uint8)

In [ ]:
def collect(fn):
    archive = zipfile.ZipFile(fn, 'r')
    annotated_images = []

    annotation_files = []
    for f in archive.namelist():
        if "det/det.txt" in f:
            annotation_files.append(f)

    image_annotations = collections.defaultdict(list)

    for af in annotation_files:
        st, sample,_,__ = af.split("/")
        imgpath = "%s/%s/img1/%06d.jpg"
        for row in csv.reader(TextIOWrapper(archive.open(af), 'utf-8') ):
            frame, _, x, y, w, h = [int(x) for x in row[:6]]
            ifn = imgpath%(st,sample, frame)
            image_annotations[ifn].append((x,y,w,h))

    for k in image_annotations.keys():
        img = cv2.imdecode(get_np_array_from_tar_object(archive.open(k)), 1 )
        for box in image_annotations[k]:
            x,y,w,h = box
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0), 2)

        img = cv2.resize(img, (0,0), fx=0.75, fy=0.75)
        cv2.imshow('image',img)
        cv2.waitKey(100)

In [ ]:
train = collect("./data/tracking/train.zip")
cv2.destroyAllWindows()

In [ ]:
current_dir = os.getcwd()

In [ ]:
# ４ types of labels are used.
labels = ['ball', 'player', 'referee', 'goalkeeper']
label_dict = {'ball': 0, 'player': 1, 'referee': 2, 'goalkeeper': 3, 'goalkeepers': 3}

In [ ]:
# change the directory
train_dirs = sorted(glob(current_dir + '/data/tracking/train/SNMOT*'))
valid_dirs = train_dirs[50:]
train_dirs = train_dirs[:50]

debug=True
if debug:
    train_dirs = train_dirs[:10]
    valid_dirs = valid_dirs[:10]


split_dirs = {
    'train':train_dirs,
    'valid':valid_dirs

}

In [ ]:
split_dirs

In [ ]:
#(1) image file path
# change directories
yolo_train_img_dir = current_dir + '/yolov7/images/train'
yolo_valid_img_dir = current_dir + '/yolov7/images/valid'

#(2) label file path
yolo_train_label_dir = current_dir + '/yolov7/labels/train'
yolo_valid_label_dir = current_dir + '/yolov7/labels/valid'

#(3) config file path
yaml_file = current_dir + '/yolov7/data.yaml'

os.makedirs(yolo_train_img_dir, exist_ok=True)
os.makedirs(yolo_valid_img_dir, exist_ok=True)
os.makedirs(yolo_train_label_dir, exist_ok=True)
os.makedirs(yolo_valid_label_dir, exist_ok=True)

In [ ]:
# convert from x,y,w,h to yolo format
def get_yolo_format_bbox(img_w, img_h, box):
    w = box[2]
    h = box[3]
    xc = box[0] + int(np.round(w/2))
    yc = box[1] + int(np.round(h/2))
    box = [xc/img_w, yc/img_h, w/img_w, h/img_h]
    box = [f"{i:.4g}" for i in box]
    return box

# get SoccerNet label info
def get_info(info):
    results = []
    for line in open(info):
        m = re.match('trackletID_(\d+)= (\S*).*', line.replace(';', ' '))
        if m:
            if m.group(2) not in label_dict:
                #print('bad label:', m.group(2))
                continue
            results.append([m.group(1), m.group(2)])
    return pd.DataFrame(results, columns=['id','lbl']).set_index('id').to_dict()['lbl']

# make image simlink and label files
for split in split_dirs:
    if split == 'train':
        yolo_img_dir = yolo_train_img_dir
        yolo_label_dir = yolo_train_label_dir
    else:
        yolo_img_dir = yolo_valid_img_dir
        yolo_label_dir = yolo_valid_label_dir

    for this_dir in split_dirs[split]:
        video = this_dir.split('/')[-1]
        info = this_dir + '/gameinfo.ini'
        det = this_dir + '/gt/gt.txt'
        info_dict = get_info(info)
        det_df = pd.read_csv(det, names=['frame','player','x','y','w','h','f1','f2','f3','f4'], usecols=['frame','player','x','y','w','h'])
        det_df['label'] = det_df.player.astype(str).map(info_dict)
        det_df['label_id'] = det_df['label'].map(label_dict)
        # check
        ng_list = list(det_df[det_df.label_id.isnull()].label.unique())
        if len(ng_list)>0:
            #print('ng_list:',ng_list, det_df.dropna().shape, det_df.shape)
            det_df = det_df.dropna()
        for grp, grp_df in det_df.groupby('frame'):
            frame = f'{grp:06}'
            img_file = f'{this_dir}/img1/{frame}.jpg'
            dst_file = f'{yolo_img_dir}/{video}_{frame}.jpg'
            if not os.path.exists(dst_file):
                os.symlink(img_file, dst_file)
            img = cv2.imread(dst_file)
            height, width, _ = img.shape
            bboxes = []
            for arr in grp_df[['x', 'y', 'w', 'h', 'label_id']].values:
                box = get_yolo_format_bbox(width, height, arr[:4])
                bboxes.append([arr[4]]+box)
            file_name = f'{yolo_label_dir}/{video}_{frame}.txt'
            with open(file_name, 'w') as f:
                for i, bbox in enumerate(bboxes):
                    bbox = [str(i) for i in bbox]
                    str_bbox = ' '.join(bbox)
                    f.write(str_bbox)
                    f.write('\n')

In [ ]:
# Dump config file
data_yaml = dict(
    train = yolo_train_img_dir,
    val = yolo_valid_img_dir,
    nc = 4,
    names = labels
)

with open(yaml_file, 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)

Move to the parent directory in yolo7 folder and use the cmd:

`time python3 train_aux.py     --workers 6     --batch-size 10     --data data.yaml     --img 640     --epochs 20     --cfg cfg/training/yolov7-w6.yaml     --weights yolov7-w6_training.pt --device cpu    --name yolov7-w6-custom     --hyp data/hyp.scratch.custom.yaml`

Choose the batch size, number of epochs etc.


In [ ]:
# deploy in the localhost:8080 a monitor board that can load every run
%load_ext tensorboard
%tensorboard --logdir "directory of yolo7/runs" --host localhost --port 8080